In [1]:
# Part 1
import math
from copy import deepcopy

def map_to_points(asteroids):
    points = []
    for y, row in enumerate(asteroids.splitlines()):
        for x, postion in enumerate(row):
            if postion == "#":
                points.append((x, y))
    
    return points

def get_degrees(origin, target):
    ox, oy = origin
    tx, ty = target
    
    degrees = math.degrees(math.atan2(oy - ty, ox - tx))
    
    degrees -= 90

    if degrees < 0:
        degrees = degrees + 360
        
    return degrees

def get_distance(origin, target):
    ox, oy = origin
    tx, ty = target
    return math.sqrt(abs(ox - tx)**2 + abs(oy - ty)**2)

def count_lines(origin, points):
    
    other_points = deepcopy(points)
    other_points.remove(origin)
    
    angles = set()
    
    for point in other_points:
        angles.add(get_degrees(origin, point))
        
    return len(angles)

def find_best_base(points):
    best_count = 0
    best_loc = None
    for point in points:
        count_visible = count_lines(point, points)
        if count_visible > best_count:
            best_count = count_visible
            best_loc = point
            
    return best_count, best_loc

asteroids = open('input/day10.txt').read()
points = map_to_points(asteroids)
find_best_base(points)

(303, (26, 29))

In [2]:
from collections import OrderedDict

def get_asteroids_by_degree(origin, points):

    other_points = deepcopy(points)
    other_points.remove(origin)
    
    asteroids = {}
    
    for point in other_points:
        angle = get_degrees(origin, point)
        dist = get_distance(origin, point)
        if angle not in asteroids:
            asteroids[angle] = []
        asteroids[angle].append((point, dist))
        
    ordered = sorted(asteroids.items()) # sort by angle
    return [(angle, sorted(asteroid, key=lambda ast: ast[1])) for angle, asteroid in ordered]

def calc_200th_eliminated(sorted_angles):
    ast_cnt = 0
    i = 0
    last_asteroid = None
    while ast_cnt < 200:
        _, asteroids = sorted_angles[i]
        last_asteroid = asteroids.pop(0)
#         print(f'{i}: {last_asteroid}')
        sorted_angles[i] = (_, asteroids)
        
        ast_cnt += 1
        i += 1
        
    return last_asteroid

sorted_angles = get_asteroids_by_degree((26, 29), points)
calc_200th_eliminated(sorted_angles)

((4, 8), 30.4138126514911)